Example taken from [https://towardsdatascience.com/question-answering-with-a-fine-tuned-bert-bc4dafd45626](https://towardsdatascience.com/question-answering-with-a-fine-tuned-bert-bc4dafd45626)

In [3]:
# !pip install transformers

In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

2021-12-02 16:06:53.789702: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
# coqa.head()

In [5]:
# coqa.style

In [6]:
# #required columns in our dataframe
# cols = ["text","question","answer"]
# #list of lists to create our dataframe
# comp_list = []
# for index, row in coqa.iterrows():
#     for i in range(len(row["data"]["questions"])):
#         temp_list = []
#         temp_list.append(row["data"]["story"])
#         temp_list.append(row["data"]["questions"][i]["input_text"])
#         temp_list.append(row["data"]["answers"][i]["input_text"])
#         comp_list.append(temp_list)
# new_df = pd.DataFrame(comp_list, columns=cols) 
# #saving the dataframe to csv file for further loading
# new_df.to_csv("data/CoQA_data.csv", index=False)

In [2]:
data = pd.read_csv("data/CoQA_data.csv")
data.head()

,text,question,answer
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law"
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology"
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project


In [3]:
print("Number of question and answers: ", len(data))

Number of question and answers:  108647


In [6]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [7]:
random_num = np.random.randint(0,len(data))
question = data["question"][random_num]
text = data["text"][random_num]

In [8]:
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 401 tokens.


In [9]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:16}{:16,}'.format(token,id))

how                        2,129
many                       2,116
followers                  8,771
did                        2,106
they                       2,027
have                       2,031
in                         1,999
2008                       2,263
?                          1,029
(                          1,006
cnn                       13,229
)                          1,007
-                          1,011
-                          1,011
cnn                       13,229
marked                     4,417
a                          1,037
milestone                 19,199
monday                     6,928
.                          1,012
just                       2,074
before                     2,077
noon                      11,501
,                          1,010
the                        1,996
@                          1,030
cnn                       13,229
##br                      19,892
##k                        2,243
account                    4,070
topped    

In [10]:
#first occurence of [SEP] token
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)
#number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)
#number of tokens in segment B (text)
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)
#creating the segment ids
segment_ids = [0]*num_seg_a + [1]*num_seg_b
#making sure that every input token has a segment id
assert len(segment_ids) == len(input_ids)

ValueError: 102 is not in list

In [ ]:
#token input_ids to represent the input and token segment_ids to differentiate our segments - question and text
output = model(torch.tensor([input_ids]),  token_type_ids=torch.tensor([segment_ids]))

NameError: name 'segment_ids' is not defined

In [ ]:
#tokens with highest start and end scores
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")
    
print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))


Question:
When did the vita get released in the us?

Answer:
2005.


In [ ]:
answer = tokens[answer_start]
for i in range(answer_start+1, answer_end+1):
    if tokens[i][0:2] == "##":
        answer += tokens[i][2:]
    else:
        answer += " " + tokens[i]

NameError: name 'answer_start' is not defined

In [ ]:
def question_answer(question, text):
    
    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1
    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    print("\nPredicted answer for '{0}':\n{1}".format(question,answer.capitalize()))

In [ ]:
text = """
New York (CNN) -- More than 80 Michael Jackson collectibles -- including the late pop star's famous rhinestone-studded 
glove from a 1983 performance -- were auctioned off Saturday, reaping a total $2 million. Profits from the auction at 
the Hard Rock Cafe in New York's Times Square crushed pre-sale expectations of only $120,000 in sales. The highly 
prized memorabilia, which included items spanning the many stages of Jackson's career, came from more than 30 fans, 
associates and family members, who contacted Julien's Auctions to sell their gifts and mementos of the singer. 
Jackson's flashy glove was the big-ticket item of the night, fetching $420,000 from a buyer in Hong Kong, China. 
Jackson wore the glove at a 1983 performance during \"Motown 25,\" an NBC special where he debuted his revolutionary 
moonwalk. Fellow Motown star Walter \"Clyde\" Orange of the Commodores, who also performed in the special 26 years 
ago, said he asked for Jackson's autograph at the time, but Jackson gave him the glove instead. "The legacy that 
[Jackson] left behind is bigger than life for me,\" Orange said. \"I hope that through that glove people can see what 
he was trying to say in his music and what he said in his music.\" Orange said he plans to give a portion of the 
proceeds to charity. Hoffman Ma, who bought the glove on behalf of Ponte 16 Resort in Macau, paid a 25 percent buyer's 
premium, which was tacked onto all final sales over $50,000. Winners of items less than $50,000 paid a 20 percent premium.
"""
question = "Where was the Auction held?"
question_answer(question, text)
#original answer from the dataset
print("Original answer:\n", data.loc[data["question"] == question]["answer"].values[0])


Predicted answer for 'Where was the Auction held?':
Hard rock cafe in new york ' s times square
Original answer:
 Hard Rock Cafe


News taken from [https://www.swissinfo.ch/eng/anti-vax-protesters-in-rome-target-draghi-s-office--union-s-headquarters/47015884](https://www.swissinfo.ch/eng/anti-vax-protesters-in-rome-target-draghi-s-office--union-s-headquarters/47015884)

In [ ]:
text = """
Anti-vax protesters in Rome target Draghi's office, union's headquarters.

Italian police used water cannon and tear gas on Saturday to push back hundreds of people, including 
neo-fascist activists, demonstrating in Rome against a government drive to make the COVID-19 "Green Pass" 
mandatory for all workers.
One group of protesters tried to break through police lines to reach Prime Minister Mario Draghi's city centre 
office, while a separate group tried to smash their way into the headquarters of Italy's main CGIL trade union.
The protests come days before Italy becomes the first country in Europe to make all workers carry the 
Green Pass in an effort to accelerate vaccinations and stamp out coronavirus infections.
The pass, a digital or paper certificate showing someone has received at least one vaccine dose, has tested 
negative or recently recovered from the virus, was originally conceived to ease travel among European Union states.
Draghi denounced Saturday's violence and said his broad unity government remained committed to completing its 
vaccination campaign.
"The right to demonstrate to support one's ideas can never degenerate into acts of aggression and intimidation," 
he said in a statement issued by his office.
Opponents of the Green Pass say it tramples on freedoms and is a back-door means of forcing people to vaccinate. 
Their cause has been backed by far-right neo-fascist groups who local politicians accused of 
orchestrating Saturday's violence.
Local media reported that around 10,000 people took to the streets of the Italian capital, with many 
chanting "freedom, freedom" as some looked to break past police in riot gear deployed to guard access 
to Draghi's office.
CGIL, which has accepted the Green Pass system for workers, condemned the attack on its offices.
"The assault on CGIL's national headquarters is an act of fascist thuggery, an attack on democracy and on 
the world of work," its leader Maurizio Landini said in a statement. "No-one should think that they can return 
our country to its fascist past.
"""
question = "Is green pass only digital?"
question_answer(question, text)
#NO original answer from the dataset - OUT OF SET


Predicted answer for 'Is green pass only digital?':
A digital or paper certificate showing someone has received at least one vaccine dose , has tested negative or recently recovered from the virus


In [ ]:
question = "Who is Draghi?"
question_answer(question, text)


Predicted answer for 'Who is Draghi?':
Prime minister


In [ ]:
question = "Why people attacked the trade union?"
question_answer(question, text)


Predicted answer for 'Why people attacked the trade union?':
Fascist thuggery


In [ ]:
question = "What's the name of the trade union attacked?"
question_answer(question, text)


Predicted answer for 'What's the name of the trade union attacked?':
Cgil


In [ ]:
question = "Why is the green pass an attack to freedom?"
question_answer(question, text)


Predicted answer for 'Why is the green pass an attack to freedom?':
It tramples on freedoms and is a back - door means of forcing people to vaccinate


Another Example

from [https://www.britannica.com/topic/The-Old-Man-and-the-Sea-novel-by-Hemingway](https://www.britannica.com/topic/The-Old-Man-and-the-Sea-novel-by-Hemingway)

In [ ]:
text2 = """
The Old Man and the Sea, short heroic novel by Ernest Hemingway, published in 1952 and awarded the 1953 
Pulitzer Prize for fiction. It was his last major work of fiction. The story centres on an aging fisherman 
who engages in an epic battle to catch a giant marlin.

The central character is an old Cuban fisherman named Santiago, who has not caught a fish for 84 days. 
The family of his apprentice, Manolin, has forced the boy to leave the old fisherman, though Manolin 
continues to support him with food and bait. Santiago is a mentor to the boy, who cherishes the old man and 
the life lessons he imparts. Convinced that his luck must change, Santiago takes his skiff far out into 
the deep waters of the Gulf Stream, where he soon hooks a giant marlin. With all his great experience 
and strength, he struggles with the fish for three days, admiring its strength, dignity, and faithfulness 
to its identity; its destiny is as true as Santiago’s as a fisherman. He finally reels the marlin in and 
lashes it to his boat.
"""

In [ ]:
question2a = "What is the story called?"
question_answer(question2a, text2)

question2b = "What is the story about?"
question_answer(question2b, text2)


Predicted answer for 'What is the story called?':
The old man and the sea

Predicted answer for 'What is the story about?':
An aging fisherman who engages in an epic battle to catch a giant marlin


In [ ]:
question2c = "What was his last work of fiction?"
question_answer(question2c, text2)

question2d = "What was her last work of fiction?"
question_answer(question2d, text2)


Predicted answer for 'What was his last work of fiction?':
The old man and the sea

Predicted answer for 'What was her last work of fiction?':
The old man and the sea


In [ ]:
question2e = "What is the size of the universe according to Hemingway?"
question_answer(question2e, text2)


Predicted answer for 'What is the size of the universe according to Hemingway?':
84 days


In [ ]:
question2f = "What is the age of the universe?"
question_answer(question2f, text2)


Predicted answer for 'What is the age of the universe?':
84 days


In [ ]:
question2g = "Why is Manolin supporting Santiago?"
question_answer(question2g, text2)


Predicted answer for 'Why is Manolin supporting Santiago?':
Food and bait


In [12]:
#from transformers import BlenderbotSmallTokenizer, BlenderbotSmallForConditionalGeneration
#mname = 'facebook/blenderbot_small-90M'
#model = BlenderbotSmallForConditionalGeneration.from_pretrained(mname)
#tokenizer = BlenderbotSmallTokenizer.from_pretrained(mname)

from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
#mname = 'facebook/blenderbot-90M'
mname = 'facebook/blenderbot-400M-distill'
#mname = 'facebook/blenderbot-1B-distill'
#mname = 'facebook/blenderbot-3B' 
model = BlenderbotForConditionalGeneration.from_pretrained(mname)
tokenizer = BlenderbotTokenizer.from_pretrained(mname)

ImportError: cannot import name 'BlenderbotTokenizer' from 'transformers' (/Users/davebrunner/opt/anaconda3/lib/python3.8/site-packages/transformers/__init__.py)

In [ ]:
def wrap_interaction(entry):
    return "<s>"+entry+"</s>"

def interact_with_BOT(interaction, query):
    print("Previous interactions: ")
    nr = int(interaction.count('</s>')/2)
    if (nr > 0):
        for i in interaction.split("</s><s>"):
            print(" --> "+i.replace("<s>","").replace("</s>","")) 
    else:
        print("--- NONE ---")
    print("Interaction #{}".format(nr+1))
    print(" Human: ", query)
    interaction += wrap_interaction(query)
    inputs = tokenizer([interaction], return_tensors='pt')
    next_reply_ids = model.generate(**inputs)
    reply = tokenizer.batch_decode(next_reply_ids, skip_special_tokens=True)[0]
    print(" Bot: ", reply)
    interaction += wrap_interaction(reply) 
    return interaction

In [ ]:
step0 = interact_with_BOT("","My dad is not answering")

Previous interactions: 
--- NONE ---
Interaction #1
 Human:  My dad is not answering
 Bot:   I'm sorry to hear that. I hope he comes back soon. What's going on?


In [ ]:
step1 = interact_with_BOT(step0,"No, he seems to be unconscious...")

Previous interactions: 
 --> My dad is not answering
 -->  I'm sorry to hear that. I hope he comes back soon. What's going on?
Interaction #2
 Human:  No, he seems to be unconscious...
 Bot:   I don't know what to do. I feel like I'm going to have to call the police.


In [ ]:
step2 = interact_with_BOT(step1,"This is the issue... Can you think about something else, maybe?")

Previous interactions: 
 --> My dad is not answering
 -->  I'm sorry to hear that. I hope he comes back soon. What's going on?
 --> No, he seems to be unconscious...
 -->  I don't know what to do. I feel like I'm going to have to call the police.
Interaction #3
 Human:  This is the issue... Can you think about something else, maybe?
 Bot:   I can't think of anything else at the moment. I just want him to come home.


In [ ]:
step0 = interact_with_BOT("","My PC does not start up")

Previous interactions: 
--- NONE ---
Interaction #1
 Human:  My PC does not start up
 Bot:   I'm sorry to hear that. I hope it gets fixed soon. What's wrong with it?


In [ ]:
step1 = interact_with_BOT(step0,"I think the reson is the operating system")

Previous interactions: 
 --> My PC does not start up
 -->  I'm sorry to hear that. I hope it gets fixed soon. What's wrong with it?
Interaction #2
 Human:  I think the reson is the operating system
 Bot:   I am not sure what is wrong. I am going to have to take it in to get it fixed.


In [ ]:
step2 = interact_with_BOT(step1,"What operating system do you think it has?")

Previous interactions: 
 --> My PC does not start up
 -->  I'm sorry to hear that. I hope it gets fixed soon. What's wrong with it?
 --> I think the reson is the operating system
 -->  I am not sure what is wrong. I am going to have to take it in to get it fixed.
Interaction #3
 Human:  What operating system do you think it has?
 Bot:   I don't know, but it's been acting up for a few days now.


In [ ]:
step3 = interact_with_BOT(step2,"I don't want to leave my PC here...")

Previous interactions: 
 --> My PC does not start up
 -->  I'm sorry to hear that. I hope it gets fixed soon. What's wrong with it?
 --> I think the reson is the operating system
 -->  I am not sure what is wrong. I am going to have to take it in to get it fixed.
 --> What operating system do you think it has?
 -->  I don't know, but it's been acting up for a few days now.
Interaction #4
 Human:  I don't want to leave my PC here...
 Bot:   That's too bad.  I hope you can get it sorted out soon.
